# Word2Vec
## Imports

In [29]:
import numpy as np
import pandas as pd
import pickle
import random
import spacy

from gensim.models import Word2Vec
from itertools import chain
from pathlib import Path
from sklearn.metrics.pairwise import cosine_similarity

## Constant paths

In [10]:
# Path to directory containing preprocessed COCA files
COCA_PREPROC_DIR = Path("/Volumes/Elements/Capstone/coca-preproc-spacy/")

# Path to directory containing preprocessed Elsevier files
ELSEVIER_PREPROC_DIR = Path("/Volumes/Elements/Capstone/elsevier-preproc-spacy/")

# Path to file containing all subject areas
SUBJAREAS = Path("./subjareas.txt")

## Selecting Comparison Words
First, we select 1000 tokens that are present in all corpora. These will be used to measure the distance between the general language corpus and the academic corpora.

In [11]:
nlp = spacy.load("en_core_web_sm")
stopwords = nlp.Defaults.stop_words

In [4]:
unique_tokens = set()

with open(f'{COCA_PREPROC_DIR}/2015.pickle', 'rb') as f:
    coca = pickle.load(f)

    for text in coca:
        text_tokens = [
            token.text for sentence in text['text_docs'] for token in sentence
            ]
        filter_tokens = [
            token.lower() for token in text_tokens if token.lower() not in stopwords
            ]

        unique_tokens.update(filter_tokens)

with open(f'{SUBJAREAS}', 'r') as subj_list_file:
    for subject in subj_list_file:
        subject = subject.strip()

        with open(f'{ELSEVIER_PREPROC_DIR}/{subject}.pickle', 'rb') as articles_file:
            articles = pickle.load(articles_file)

        unique_subj_tokens = set()

        for text in articles:
            text_tokens = [
                token.text for sentence in text['body_text_docs'] for token in sentence if token.text
                ]
            filter_tokens = [
                token.lower() for token in text_tokens if token.lower() not in stopwords
                ]

            unique_subj_tokens.update(filter_tokens)
        
        unique_tokens = unique_tokens.intersection(unique_subj_tokens)

tokens_1000 = random.sample(unique_tokens, k=1000)
        

In [43]:
tokens_1000 = random.sample(unique_tokens, k=1000)
tokens_1000[:15]

['demographic',
 'database',
 'discernible',
 'sufficiently',
 'nonetheless',
 'assuming',
 'irrelevant',
 'let',
 'ongoing',
 'abstraction',
 'landmarks',
 'augmented',
 'shear',
 'helped',
 'trends']

## COCA Embedding Space
First we create an embedding space based on COCA, and save all the word vectors from the 1000 randomly selected tokens in a list.

In [18]:
coca_sentences = [
    [token.text.lower() for token in sentence if token.text.lower() not in stopwords] 
    for text in coca for sentence in text['text_docs']
    ]

coca_model = Word2Vec(coca_sentences, window=5, min_count=1, workers=4)
coca_token_vectors = [(token, coca_model.wv[token]) for token in tokens_1000]


## ElSevier OA CC-BY Embedding Space
Next, we create an embedding space for each academic corpus and compare the word vectors in the academic corpora with the word vectors in the COCA embedding space. This procedure is bootstrapped with 100 iterations.

In [30]:
def subj_vector_sim(articles, vector_token_list):
    """
    Given a set of articles and a list of word vectors, create a word2vec embedding space
    based on articles and return the average cosine similarity between each word vector in 
    vector_token_list and their word vector in the new embedding space.

    Args:
    articles: Preprocessed articles from a specific discipline in the 
    ElSevier OA CC-BY corpus.
    vector_token_list: A list of tuples. The item in the tuple contains a token, and the second 
    item in contains the word vector for that token in the COCA embedding space.
    """

    subj_sentences = [
        [token.text.lower() for token in sentence if token.text.lower() not in stopwords] 
        for text in articles for sentence in text['body_text_docs']
        ]
    
    all_tokens = list(chain.from_iterable(subj_sentences))

    subj_model = Word2Vec(subj_sentences, window=5, min_count=1, workers=4)

    cosine_similarities = [
        cosine_similarity(subj_model.wv[token[0]].reshape(1, -1), token[1].reshape(1, -1))
        for token in vector_token_list if token[0] in all_tokens
        ]

    avg_cosine_similarity = np.mean(cosine_similarities)

    return avg_cosine_similarity

def bootstrap_word_embed_ci(
        subj_articles,
        word_vector_token_list, 
        num_resamples=100, 
        alpha=0.05, 
        ):
    """
    Bootstrap the vector cosine similarity procedure to generate an average cosine similarity
    with confidence intervals.

    Args:
    subj_articles: Preprocessed articles from a specific discipline in the 
    ElSevier OA CC-BY corpus.
    word_vector_token_list: A list of tuples. The item in the tuple contains a token, and the second 
    item in contains the word vector for that token in the COCA embedding space.
    num_resamples: The number of iterations for bootstrappign. Default = 100.
    alpha: The alpha value for which to calculate the confidence intervals. Default = 0.05.
    """
    average_cosine_sim = []
    n = len(subj_articles)

    for i in range(num_resamples):
        resampled_texts = np.random.choice(subj_articles, size=n, replace=True)

        sample_vector_sim = subj_vector_sim(resampled_texts, word_vector_token_list)

        average_cosine_sim.append(sample_vector_sim)

    average_sim = np.mean(average_cosine_sim)    
    ci_lower = np.percentile(average_cosine_sim, alpha/2 * 100)
    ci_upper = np.percentile(average_cosine_sim, (1 - alpha/2) * 100)

    return average_sim, ci_lower, ci_upper


In [37]:
# Dataframe to save results
avg_cosine_sim_df = pd.DataFrame(columns=['subj', 'avg', 'lower CI', 'upper CI'])

In [38]:
with open(f'{ELSEVIER_PREPROC_DIR}/DENT.pickle', 'rb') as articles_file:
    articles = pickle.load(articles_file)

    bootstrap_word_vec_sim = bootstrap_word_embed_ci(articles, coca_token_vectors, num_resamples=1)
    subject_row = {
        'subj': 'DENT', 'avg': bootstrap_word_vec_sim[0], 
        'lower CI': bootstrap_word_vec_sim[1], 'upper CI': bootstrap_word_vec_sim[2]
    }
            
    avg_cosine_sim_df = avg_cosine_sim_df.append(subject_row, ignore_index=True)

In [46]:
avg_cosine_sim_df.sort_values(by="avg", ignore_index=True)

,subj,avg,lower CI,upper CI
0,CHEM,0.975000,0.973609,0.976081
1,CENG,0.975214,0.974769,0.976625
2,DENT,0.975412,0.975137,0.976084
3,PHYS,0.976533,0.976112,0.977410
4,PHAR,0.977424,0.976196,0.977934
5,AGRI,0.977696,0.977221,0.978083
6,IMMU,0.978124,0.977284,0.979814
7,ENVI,0.979977,0.979750,0.981548
8,BIOC,0.980124,0.978264,0.981100
9,NEUR,0.982154,0.980366,0.983440
